# HW-SW Cosim (Example)

- 소프트웨어가 특정 명령을 실행하면 하드웨어 블록(LED 제어기)이 반응하는 과정을 시뮬레이션으로 확인.
- SW → HW → 결과 신호 흐름을 직관적으로 이해.
- 필요 환경
  - 시뮬레이션 툴: Icarus verilog (iverilog), ModelSim, Verilator, 또는 VCS
  - 하드웨어 블록: 간단한 LED Controller (Verilog/VHDL)
  - 소프트웨어 실행 모델: Instruction Set Simulator(ISS) 또는 단순 BFM(Testbench)

## Tool setup

- Icarus Verilog (iverilog):
  - 장점:
    - 설치가 간단하고 Colab에서 바로 apt-get install iverilog로 사용 가능.
    - 학습용으로 적합: Verilog 문법 지원이 넓고, 간단한 Testbench 실행에 충분.
    - 출력 파형을 vcd 파일로 저장해 GTKWave 같은 툴로 시각화 가능.
  - 단점:
    - 속도가 느리고, 대규모 설계에는 비효율적.
    - SystemVerilog 지원이 제한적.
- Verilator
  - 장점:
    - 매우 빠른 시뮬레이션: Verilog 코드를 C++로 변환해 실행하므로 대규모 설계에 유리.
    - 오픈소스 프로젝트에서 널리 사용, 산업 현장에서도 활용.
    - SystemVerilog 일부 지원.
  - 단점:
    - 설치와 사용법이 조금 더 복잡함.
    - 단순한 교육용 예제(LED 켜기/끄기 같은 간단한 Testbench)에는 오히려 과할 수 있음.

- GTKwave:
  - vcd(value change dump) 파일 wave viewer
  - https://sourceforge.net/projects/gtkwave/ 또는 여러 인터넷 사이트에서 다운로드 설치 가능 (Windows version)

In [1]:
# Icarus Verilog 설치
!apt-get update
!apt-get install -y iverilog
!apt-get install -y iverilog vvp iverilog-vpi

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,860 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,573 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/multiverse amd64 Packages [69.2 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [6,411 kB]
Get:14 http:

## How to use iverilog
- A simple led controller (simple testbench)

In [35]:
%%writefile led_controller.v

module led_controller (
    input wire clk,
    input wire reset,
    input wire [7:0] cmd,
    output reg led
);

always @(posedge clk or posedge reset) begin
    if (reset) begin
        led <= 0;
    end else begin
        case (cmd)
            8'h01: led <= 1;   // LED ON
            8'h00: led <= 0;   // LED OFF
            default: led <= led;
        endcase
    end
end

endmodule


Overwriting led_controller.v


In [39]:
%%writefile tb_led_controller.v

module tb_led_controller;
    reg clk, reset;
    reg [7:0] cmd;
    wire led;

    led_controller uut (
        .clk(clk),
        .reset(reset),
        .cmd(cmd),
        .led(led)
    );

    always #5 clk = ~clk;

    initial begin
        $dumpfile("led.vcd");
        $dumpvars(0, tb_led_controller);

        clk = 0; reset = 1; cmd = 8'h00;
        #10 reset = 0;

        // Test 1: LED ON
        #10 cmd = 8'h01;
        #10 if (led !== 1) $display("ERROR: LED should be ON");

        // Test 2: LED OFF
        #20 cmd = 8'h00;
        #10 if (led !== 0) $display("ERROR: LED should be OFF");

        // Test 3: LED 유지
        #20 cmd = 8'hFF; // 정의되지 않은 명령어
        #10 if (led !== 0) $display("ERROR: LED should remain OFF");

        $display("Simulation finished. Verification complete.");
        #20 $finish;
    end
endmodule


Overwriting tb_led_controller.v


In [42]:
!ls


cmd.txt      hello_vpi.vpi     led.vcd	       read_cmd_vpi.vpi
hello.v      hello.vvp	       Makefile        sample_data
hello_vpi.c  led_controller.v  read_cmd_vpi.c  tb_led_controller.v
hello_vpi.o  led_sim	       read_cmd_vpi.o  tb_sim


In [41]:
# compile
!iverilog -o led_sim tb_led_controller.v led_controller.v
# 실행
!vvp led_sim
# vcd 파일 download 하여 GTKWave 이용하여 view

VCD info: dumpfile led.vcd opened for output.
Simulation finished. Verification complete.


## LED Controller + VPI 예제 (개념)
  - 순수 testbench 방식
    - 위에서는 Verilog 내부에서 if (led !== expected) 같은 구문으로 결과를 직접 체크했다.
    - 간단하고 빠르게 동작 검증 가능하지만, 외부 환경과의 연동은 불가능, Verilog 언어 안에서만 검증 가능.
  - VPI 활용 방식
    - Verilog 시뮬레이터와 외부 C 코드 연결.
    - 외부 파일에서 테스트 벡터(cmd 값)를 읽어와 Verilog에 전달.
    - 시뮬레이션 중 LED 출력 값을 C 코드로 가져와 로그/분석.
    - 복잡한 검증 로직을 C로 작성해 Verilog와 함께 실행 가능.
    - HW-SW coverification을 더 현실적으로 시뮬레이션 가능.

In [28]:
%%writefile tb_led_controller.v

`timescale 1ns/1ps

module tb_led_controller;
    reg clk, reset;
    reg [7:0] cmd;
    wire led;

    // DUT 연결
    led_controller uut (
        .clk(clk),
        .reset(reset),
        .cmd(cmd),
        .led(led)
    );

    // Clock 생성
    always #5 clk = ~clk;

    integer i;
    initial begin
        $dumpfile("led.vcd");
        $dumpvars(0, tb_led_controller);

        // 초기화
        clk = 0; reset = 1; cmd = 8'h00;
        #10 reset = 0;

        // cmd.txt에서 여러 줄 읽어오기
        for (i = 0; i < 4; i = i + 1) begin
            #10 $read_cmd_from_file(cmd);   // VPI system task 호출
            #10 $display("Cycle %0d: cmd=%h, led=%b", i, cmd, led);
        end

        #20 $finish;
    end
endmodule




Overwriting tb_led_controller.v


- C 코드 (read_cmd_vpi.c)

In [29]:
%%writefile read_cmd_vpi.c

#include <vpi_user.h>
#include <stdio.h>

static FILE *fp = NULL;

PLI_INT32 read_cmd_calltf(char*user_data) {
    vpiHandle arg_iter, arg_handle;
    s_vpi_value val;

    // 첫 호출 시 파일 열기
    if (fp == NULL) {
        fp = fopen("cmd.txt", "r");
        if (!fp) {
            vpi_printf("VPI ERROR: cannot open cmd.txt\n");
            return 0;
        }
    }

    // 인자로 전달된 cmd 핸들 가져오기
    arg_iter = vpi_iterate(vpiArgument, vpi_handle(vpiSysTfCall, NULL));
    arg_handle = vpi_scan(arg_iter);

    // 한 줄씩 읽기 (파일 포인터는 자동으로 다음 줄로 이동)
    int cmd_val;
    if (fscanf(fp, "%x", &cmd_val) == 1) {
        val.format = vpiIntVal;
        val.value.integer = cmd_val;
        vpi_put_value(arg_handle, &val, NULL, vpiNoDelay);
        vpi_printf("VPI: cmd loaded = 0x%02X\n", cmd_val);
    } else {
        vpi_printf("VPI: end of file reached\n");
    }

    return 0;
}

void register_read_cmd() {
    s_vpi_systf_data tf_data;
    tf_data.type = vpiSysTask;
    tf_data.tfname = "$read_cmd_from_file";
    tf_data.calltf = read_cmd_calltf;
    tf_data.compiletf = 0;
    tf_data.sizetf = 0;
    tf_data.user_data = 0;
    vpi_register_systf(&tf_data);
}

void (*vlog_startup_routines[])() = {
    register_read_cmd,
    0
};


Writing read_cmd_vpi.c


In [30]:
%%writefile cmd.txt
01
00
FF
01

Writing cmd.txt


In [31]:
# !gcc -fPIC -shared -o read_cmd_vpi.vpi read_cmd_vpi.c
#
!iverilog-vpi read_cmd_vpi.c
!iverilog -o tb_sim tb_led_controller.v led_controller.v
!vvp -M. -m read_cmd_vpi tb_sim


Compiling read_cmd_vpi.c...
read_cmd_vpi.c: In function ‘read_cmd_calltf’:
read_cmd_vpi.c:7:32: warning: unused parameter ‘user_data’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wunused-parameter-Wunused-parameter]8;;]
    7 | PLI_INT32 read_cmd_calltf(char*user_data) {
      |                           ~~~~~^~~~~~~~~
read_cmd_vpi.c: At top level:
read_cmd_vpi.c:38:6: warning: function declaration isn’t a prototype []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wstrict-prototypes-Wstrict-prototypes]8;;]
   38 | void register_read_cmd() {
      |      ^~~~~~~~~~~~~~~~~
read_cmd_vpi.c:49:1: warning: function declaration isn’t a prototype []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wstrict-prototypes-Wstrict-prototypes]8;;]
   49 | void (*vlog_startup_routines[])() = {
      | ^~~~
Making read_cmd_vpi.vpi from  read_cmd_vpi.o...
VCD info: dumpfile led.vcd opened for output.
VPI: cmd loaded = 0x01
Cycle 0: cmd=01

```
[전체 흐름 요약]
Verilog Simulator
         ↓
로드된 .vpi 파일 → vlog_startup_routines[] 실행
         ↓
register_read_cmd() → $read_cmd_from_file 등록
         ↓
Verilog Testbench에서 $read_cmd_from_file(cmd) 호출
         ↓
read_cmd_calltf() 실행 → cmd.txt에서 값 읽음
         ↓
vpi_put_value()로 Verilog 신호(cmd)에 값 전달
         ↓
DUT가 cmd에 따라 동작 → 결과 확인
```

# Exercise

## VPI/DPI

- VPI (Verilog Procedural Interface)
  - Verilog 시뮬레이터(예: Icarus Verilog)와 C 프로그램을 연결하는 인터페이스.
  - 시뮬레이션 중에 외부 C 함수 호출, 신호 값 읽기/쓰기, 이벤트 제어 가능.
  - Icarus Verilog는 VPI를 지원하므로 교육용 데모에 적합.
- DPI (Direct Programming Interface)
  - SystemVerilog에서 C/C++ 함수를 직접 호출하는 방식.
  - 더 현대적이고 직관적이지만, Icarus Verilog는 SystemVerilog DPI를 완전 지원하지 않음.
- 비교:
  - DPI: SystemVerilog에서 C/C++ 함수를 직접 호출하는 방식
  - VPI: 하드웨어 시뮬레이션 상태를 그대로 유지하면서 외부에서 조작하는 방식

## VPI Intro

- VPI (Verilog Procedural Interface): Verilog 시뮬레이터가 C 코드와 상호 작용할 수 있도록 해주는 표준
- 즉, Verilog에서 $hello_vpi 같은 사용자 정의 시스템 함수를 호출하면, 그 동작을 C 코드에서 정의할 수 있다. (user-defined system function)
- main() 이 없고 simulator 에 의해 실행될 때 로드된다.
  - C-code 가 entry point 가 아니고, 시뮬레이터 (e.g. vvp) 가 실행의 주체, 즉, main process.

## Example 1: simple testbench
- $hello_vpi는 Verilog에서 호출되는 사용자 정의 시스템 함수
- vpi_printf는 C에서 Verilog 시뮬레이터 콘솔로 출력
- vpi_register_systf로 Verilog와 C를 연결

In [44]:
%%writefile hello.v

module top;
  initial begin
    $hello_vpi;  // system function 호출
    $finish;
  end
endmodule


Overwriting hello.v


In [45]:
%%writefile hello_vpi.c

#include <vpi_user.h>
#include <stdio.h>

PLI_INT32 hello_vpi_calltf(PLI_BYTE8 *user_data) {  // C 코드에서 해당 함수 동작 정의
    vpi_printf("Hello from VPI!\n");
    return 0;
}

void hello_vpi_register() {         // vpi 등록 함수 (verilog <-> c 연결)
    s_vpi_systf_data tf_data;
    tf_data.type      = vpiSysTask;  // system task
    tf_data.tfname    = "$hello_vpi";
    tf_data.calltf    = hello_vpi_calltf;
    tf_data.compiletf = 0;
    tf_data.sizetf    = 0;
    tf_data.user_data = 0;
    vpi_register_systf(&tf_data);
}

// This array tells the simulator which functions to run at startup.

void (*vlog_startup_routines[])() = {
    hello_vpi_register,
    0
};


Overwriting hello_vpi.c


In [46]:
# compile verilog source
# vvp 파일은 실행 가능한 중간 코드(intermediate code), 즉 vvp 시뮬레이터가 읽을 수 있는 바이트코드 형태.

!iverilog hello.v -o hello.vvp

In [47]:
# build VPI module
# Verilog Procedural Interface (VPI) 모듈을 C 언어로 작성한 뒤 컴파일한 공유 라이브러리
# (.vpi 확장자를 가진 동적 라이브러리 (.so와 유사))

!iverilog-vpi hello_vpi.c
# or
# (iverilog-vpi 는 Icarus Verilog 패키지에 포함된 VPI용 컴파일러 래퍼)
# !gcc -fPIC -shared -I/usr/include/iverilog -o hello_vpi.vpi hello_vpi.c

Compiling hello_vpi.c...
hello_vpi.c: In function ‘hello_vpi_calltf’:
hello_vpi.c:5:39: warning: unused parameter ‘user_data’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wunused-parameter-Wunused-parameter]8;;]
    5 | PLI_INT32 hello_vpi_calltf(PLI_BYTE8 *user_data) {  // C 코드에서 해당 함수 동작 정의
      |                            ~~~~~~~~~~~^~~~~~~~~
hello_vpi.c: At top level:
hello_vpi.c:10:6: warning: function declaration isn’t a prototype []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wstrict-prototypes-Wstrict-prototypes]8;;]
   10 | void hello_vpi_register() {         // vpi 등록 함수 (verilog <-> c 연결)
      |      ^~~~~~~~~~~~~~~~~~
hello_vpi.c:23:1: warning: function declaration isn’t a prototype []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wstrict-prototypes-Wstrict-prototypes]8;;]
   23 | void (*vlog_startup_routines[])() = {
      | ^~~~
Making hello_vpi.vpi from  hello_vpi.o...


In [48]:
# run simulation
#  -M. -> search path for VPI modules (current directory for .vpi modules)
#  -mhello_vpi -> loads the module (hello_vpi.vpi)

!vvp -M. -mhello_vpi hello.vvp


Hello from VPI!


- 동작 흐름 요약
  - hello.v ──(iverilog)──▶ hello.vvp
  - hello_vpi.c ──(gcc/iverilog-vpi)──▶ hello_vpi.vpi
  - hello.vvp + hello_vpi.vpi ──(vvp 실행)──▶ $hello_vpi 동작


- gcc and iverilog-vpi
  - gcc:
    - 일반 C 컴파일러
    - 사용법: 직접 컴파일 옵션 지정 필요
    - 편의성: 수동 설정 필요 (-fPIC, -shared, -I 등)
    - 확장자: .so 또는 .vpi
    - 링커 필요: 직접 라이브러리 링크 필요
    - 고급 사용자, 커스텀 빌드
  - iverilog-vpi
    - VPI 모듈 전용 빌드 도구
    - 사용법: 자동으로 필요한 옵션 적용
    - 간단한 명령으로 .vpi 생성
    - 확장자: .vpi
    - 링커설정: 자동처리
    - 빠른 테스트, 표준 예제

## Example 2

In [ ]:
%%writefile add.v

module top;
  initial begin
    $add_vpi(3, 5);  // Verilog에서 C로 인자 전달
    $finish;
  end
endmodule


Writing add.v


In [ ]:
%%writefile add_vpi.c

#include <vpi_user.h>
#include <stdio.h>

PLI_INT32 add_vpi_compiletf(PLI_BYTE8 *user_data) {
    vpiHandle systf_handle = vpi_handle(vpiSysTfCall, NULL);
    vpiHandle args_iter = vpi_iterate(vpiArgument, systf_handle);

    if (args_iter == NULL) {
        vpi_printf("Error (compiletf): No arguments passed to $add_vpi.\n");
    }

    // 저장해두면 calltf에서 사용 가능
    vpi_put_userdata(systf_handle, args_iter);
    return 0;
}

PLI_INT32 add_vpi_calltf(PLI_BYTE8 *user_data) {
    vpiHandle systf_handle = vpi_handle(vpiSysTfCall, NULL);
    vpiHandle args_iter = (vpiHandle)vpi_get_userdata(systf_handle);
    vpiHandle arg;
    s_vpi_value val;
    int a, b;

    if (args_iter == NULL) {
        vpi_printf("Error (calltf): No arguments found.\n");
        return 0;
    }

    arg = vpi_scan(args_iter);
    val.format = vpiIntVal;
    vpi_get_value(arg, &val);  // 1st argument
    a = val.value.integer;

    arg = vpi_scan(args_iter);
    vpi_get_value(arg, &val);  // 2nd argument
    b = val.value.integer;

    vpi_printf("VPI Add: %d + %d = %d\n", a, b, a + b);
    return 0;
}

void add_vpi_register() {
    s_vpi_systf_data tf_data;
    tf_data.type      = vpiSysTask;
    tf_data.tfname    = "$add_vpi";
    tf_data.calltf    = add_vpi_calltf;
    tf_data.compiletf = add_vpi_compiletf;
    tf_data.sizetf    = 0;
    tf_data.user_data = 0;
    vpi_register_systf(&tf_data);
}

void (*vlog_startup_routines[])() = {
    add_vpi_register,
    0
};


Writing add_vpi.c


In [ ]:
!iverilog add.v -o add.vvp
!iverilog-vpi add_vpi.c

Compiling add_vpi.c...
add_vpi.c: In function ‘add_vpi_compiletf’:
add_vpi.c:5:40: warning: unused parameter ‘user_data’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wunused-parameter-Wunused-parameter]8;;]
    5 | PLI_INT32 add_vpi_compiletf(PLI_BYTE8 *user_data) {
      |                             ~~~~~~~~~~~^~~~~~~~~
add_vpi.c: In function ‘add_vpi_calltf’:
add_vpi.c:18:37: warning: unused parameter ‘user_data’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wunused-parameter-Wunused-parameter]8;;]
   18 | PLI_INT32 add_vpi_calltf(PLI_BYTE8 *user_data) {
      |                          ~~~~~~~~~~~^~~~~~~~~
add_vpi.c: At top level:
add_vpi.c:43:6: warning: function declaration isn’t a prototype []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wstrict-prototypes-Wstrict-prototypes]8;;]
   43 | void add_vpi_register() {
      |      ^~~~~~~~~~~~~~~~
add_vpi.c:54:1: warning: function declaration isn’t a prototype 

In [ ]:
!vvp -M. -madd_vpi add.vvp

VPI Add: 3 + 5 = 8


- gcc 방식

In [ ]:
# !gcc -fPIC -shared -Wl,-E -I/usr/include/iverilog -o add_vpi.vpi add_vpi.c
!gcc -fPIC -shared -rdynamic -Wl,-E -I/usr/include/iverilog -o add_vpi.vpi add_vpi.c

!iverilog -o simv add.v
!vvp -M. -madd_vpi add.vvp

VPI Add: 3 + 5 = 8


## Example 3 (returning a value from C)
- system function

In [ ]:
%%writefile add_func.v

module top;
  integer result;

  initial begin
    result = $add_vpi(3, 5);  // C에서 계산된 결과를 받음
    $display("Result from VPI: %d", result);
    $finish;
  end
endmodule


Overwriting add_func.v


In [ ]:
%%writefile add_func_vpi.c

#include <vpi_user.h>
#include <stdio.h>

PLI_INT32 add_vpi_compiletf(PLI_BYTE8 *user_data) {
    return 0; // 인자 핸들 저장 생략
}

PLI_INT32 add_vpi_calltf(PLI_BYTE8 *user_data) {
    vpiHandle systf_handle = vpi_handle(vpiSysTfCall, NULL);
    vpiHandle args_iter = vpi_iterate(vpiArgument, systf_handle);
    vpiHandle arg;
    s_vpi_value val;
    int a = 0, b = 0, result = 0;

    if (args_iter == NULL) {
        vpi_printf("Error: No arguments passed to $add_vpi.\n");
        return 0;
    }

    arg = vpi_scan(args_iter);
    val.format = vpiIntVal;
    vpi_get_value(arg, &val);
    a = val.value.integer;

    arg = vpi_scan(args_iter);
    vpi_get_value(arg, &val);
    b = val.value.integer;

    result = a + b;

    // 명시적으로 반환값 설정
    val.format = vpiIntVal;
    val.value.integer = result;
    vpi_put_value(systf_handle, &val, NULL, vpiNoDelay);

    return result;
}

void add_vpi_register() {
    s_vpi_systf_data tf_data;
    tf_data.type      = vpiSysFunc;         // 시스템 함수로 등록
    tf_data.tfname    = "$add_vpi";
    tf_data.calltf    = add_vpi_calltf;
    tf_data.compiletf = add_vpi_compiletf;
    tf_data.sizetf    = vpiSizedFunc;       // 반환값 크기 자동 결정
    tf_data.user_data = 0;
    vpi_register_systf(&tf_data);
}

void (*vlog_startup_routines[])() = {
    add_vpi_register,
    0
};



Overwriting add_func_vpi.c


In [ ]:
!iverilog add_func.v -o add_func.vvp
!iverilog-vpi add_func_vpi.c

Compiling add_func_vpi.c...
add_func_vpi.c: In function ‘add_vpi_compiletf’:
add_func_vpi.c:5:40: warning: unused parameter ‘user_data’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wunused-parameter-Wunused-parameter]8;;]
    5 | PLI_INT32 add_vpi_compiletf(PLI_BYTE8 *user_data) {
      |                             ~~~~~~~~~~~^~~~~~~~~
add_func_vpi.c: In function ‘add_vpi_calltf’:
add_func_vpi.c:9:37: warning: unused parameter ‘user_data’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wunused-parameter-Wunused-parameter]8;;]
    9 | PLI_INT32 add_vpi_calltf(PLI_BYTE8 *user_data) {
      |                          ~~~~~~~~~~~^~~~~~~~~
add_func_vpi.c: At top level:
add_func_vpi.c:40:6: warning: function declaration isn’t a prototype []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wstrict-prototypes-Wstrict-prototypes]8;;]
   40 | void add_vpi_register() {
      |      ^~~~~~~~~~~~~~~~
add_func_vpi.c: In function ‘a

In [ ]:
!vvp -M. -madd_func_vpi add_func.vvp

Result from VPI:           8


- comments:
  - Icarus Verilog does not support DPI (Direct Programming Interface) as of now. DPI is a feature introduced in SystemVerilog, and while Icarus Verilog has partial SystemVerilog support, it hasn't implemented DPI-C integration, which is essential for calling C/C++ functions directly from SystemVerilog.
- DPI: SystemVerilog에서 C/C++ 함수를 직접 호출하는 방식
- VPI: 하드웨어 시뮬레이션 상태를 그대로 유지하면서 외부에서 조작하는 방식

- gcc and iverilog-vpi
  - gcc:
    - 일반 C 컴파일러
    - 사용법: 직접 컴파일 옵션 지정 필요
    - 편의성: 수동 설정 필요 (-fPIC, -shared, -I 등)
    - 확장자: .so 또는 .vpi
    - 링커 필요: 직접 라이브러리 링크 필요
    - 고급 사용자, 커스텀 빌드
  - iverilog-vpi
    - VPI 모듈 전용 빌드 도구
    - 사용법: 자동으로 필요한 옵션 적용
    - 간단한 명령으로 .vpi 생성
    - 확장자: .vpi
    - 링커설정: 자동처리
    - 빠른 테스트, 표준 예제



## Example 4 (콜백 등록 방식)
- 목적: Verilog 코드에서 호출하지 않아도, 시뮬레이터 이벤트(시작, 종료, 신호 변화 등)에 맞춰 자동 실행되는 C 함수를 등록.
- 특징: “시뮬레이션 시작 시”, “종료 시” 같은 이벤트에 반응.
- 예시: 시뮬레이션 시작/종료 시 자동으로 메시지를 출력.

In [ ]:
%%writefile tb.v

module tb;
    initial begin
        #10 $finish;
    end
endmodule

Writing tb.v


In [ ]:
%%writefile callback_vpi.c

#include <vpi_user.h>
#include <stdio.h>

// 시작 시 호출
PLI_INT32 my_startup(s_cb_data *cb_data) {
    vpi_printf(">>> Simulation started!\n");
    return 0;
}

// 종료 시 호출
PLI_INT32 my_finish(s_cb_data *cb_data) {
    vpi_printf(">>> Simulation finished!\n");
    return 0;
}

// 콜백 등록
void register_callbacks() {
    s_cb_data cb_startup;
    cb_startup.reason    = cbStartOfSimulation;
    cb_startup.cb_rtn    = my_startup;
    cb_startup.obj       = NULL;
    cb_startup.time      = NULL;
    cb_startup.value     = NULL;
    cb_startup.user_data = NULL;
    vpi_register_cb(&cb_startup);

    s_cb_data cb_finish;
    cb_finish.reason    = cbEndOfSimulation;
    cb_finish.cb_rtn    = my_finish;
    cb_finish.obj       = NULL;
    cb_finish.time      = NULL;
    cb_finish.value     = NULL;
    cb_finish.user_data = NULL;
    vpi_register_cb(&cb_finish);
}

// startup table
void (*vlog_startup_routines[])() = {
    register_callbacks,
    0
};


Writing callback_vpi.c


In [ ]:
# !gcc -fPIC -shared -o callback_vpi.vpi callback_vpi.c
!iverilog-vpi callback_vpi.c
!iverilog -o tb_sim tb.v
!vvp -M. -m callback_vpi tb_sim


Compiling callback_vpi.c...
callback_vpi.c: In function ‘my_startup’:
callback_vpi.c:6:33: warning: unused parameter ‘cb_data’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wunused-parameter-Wunused-parameter]8;;]
    6 | PLI_INT32 my_startup(s_cb_data *cb_data) {
      |                      ~~~~~~~~~~~^~~~~~~
callback_vpi.c: In function ‘my_finish’:
callback_vpi.c:12:32: warning: unused parameter ‘cb_data’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wunused-parameter-Wunused-parameter]8;;]
   12 | PLI_INT32 my_finish(s_cb_data *cb_data) {
      |                     ~~~~~~~~~~~^~~~~~~
callback_vpi.c: At top level:
callback_vpi.c:18:6: warning: function declaration isn’t a prototype []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wstrict-prototypes-Wstrict-prototypes]8;;]
   18 | void register_callbacks() {
      |      ^~~~~~~~~~~~~~~~~~
callback_vpi.c:39:1: warning: function declaration isn’t a prototype []8

------------------------